# Handle imports

In [16]:
from dotenv import load_dotenv
import os
import cassio

load_dotenv()

True

In [17]:
ASTRA_DB_APPLICATION_TOKEN = os.getenv("ASTRA_DB_APPLICATION_TOKEN")
ASTRA_DB_ID = os.getenv("ASTRA_DB_ID")

cassio.init(
    token = ASTRA_DB_APPLICATION_TOKEN,
    database_id = ASTRA_DB_ID
)

In [18]:
# Build Index

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader

# Get the URLs for the recipes from the print page to reduce the extra nonesense that comes with recipes 
urls = [
    "https://www.loveandlemons.com/wprm_print/best-homemade-brownies",
    "https://www.allrecipes.com/recipe/17481/simple-white-cake/?print",
    "https://pinchofyum.com/the-best-soft-chocolate-chip-cookies/print/39213"
]


docs = [WebBaseLoader(url).load() for url in urls]
doc_list = [item for sublist in docs for item in sublist]

# Split using Characters
# text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 0)

# Split using Tokens
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size = 1000, chunk_overlap = 0)

docs_split = text_splitter.split_documents(doc_list)


In [19]:
docs_split

[Document(metadata={'source': 'https://www.loveandlemons.com/wprm_print/best-homemade-brownies', 'title': 'Best Homemade Brownies - Love and Lemons', 'language': 'en-US'}, page_content="Best Homemade Brownies - Love and Lemons     \nGo Back\nPrintRecipe ImageNotes–+\xa0servingsSmallerNormalLargerBest Homemade BrowniesPrep Time: 5 minutes minsCook Time: 45 minutes minsServes 16 browniesThe ultimate recipe for brownies! They're fudgy, moist, and super chocolaty, with perfect crispy edges. From Weeknight Baking by Michelle Lopez.Equipment8x8 Baking Dish (this is the one I use so they don't overcook)Cooking Spray (I love this avocado oil one from Chosen Foods)Parchment Paper (this makes it so much easier to remove the brownies from the pan)Ingredients1 1/2 cups granulated sugar*3/4 cup all-purpose flour2/3 cup cocoa powder, sifted if lumpy1/2 cup powdered sugar, sifted if lumpy1/2 cup dark chocolate chips3/4 teaspoons sea salt2 large eggs1/2 cup canola oil or extra-virgin olive oil**2 tabl

In [20]:
# Create the embeddings
# Chose a general purpose embedding model
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name = "all-MiniLM-L6-v2")

d:\TAMU\SEM 4\CNM\BrokenBasket\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [21]:
# Connect to the Database
from langchain.vectorstores import Cassandra
astra_vector_store = Cassandra(embedding=embeddings,
                               table_name = "CNM_test_table",
                               session=None,
                               keyspace=None)

In [22]:
# Add the documents
from langchain.indexes.vectorstore import VectorStoreIndexWrapper
astra_vector_store.add_documents(docs_split)
print(f"Inserted {len(docs_split)} headlines")
astra_vector_index = VectorStoreIndexWrapper(vectorstore=astra_vector_store)

Inserted 8 headlines


In [24]:
# test the retriever
retriever = astra_vector_store.as_retriever()
retriever.invoke("Give me a brownie recipe")

[Document(id='d61c2d48df544d9ab1ec4f56b13f4aaa', metadata={'language': 'en-US', 'source': 'https://www.loveandlemons.com/wprm_print/best-homemade-brownies', 'title': 'Best Homemade Brownies - Love and Lemons'}, page_content="Best Homemade Brownies - Love and Lemons     \nGo Back\nPrintRecipe ImageNotes–+\xa0servingsSmallerNormalLargerBest Homemade BrowniesPrep Time: 5 minutes minsCook Time: 45 minutes minsServes 16 browniesThe ultimate recipe for brownies! They're fudgy, moist, and super chocolaty, with perfect crispy edges. From Weeknight Baking by Michelle Lopez.Equipment8x8 Baking Dish (this is the one I use so they don't overcook)Cooking Spray (I love this avocado oil one from Chosen Foods)Parchment Paper (this makes it so much easier to remove the brownies from the pan)Ingredients1 1/2 cups granulated sugar*3/4 cup all-purpose flour2/3 cup cocoa powder, sifted if lumpy1/2 cup powdered sugar, sifted if lumpy1/2 cup dark chocolate chips3/4 teaspoons sea salt2 large eggs1/2 cup canol